## 今天的作業
在鐵達尼資料集中，以Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。


In [1]:
# library
import numpy as np
import pandas as pd
pd.set_option('chained_assignment', None)

# 特徵選取會用到的函數
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

## 讀入資料

In [2]:
df_train = pd.read_csv("./D36-41 探索性資料分析 (EDA)_Dataset/Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
# 1.產稱一個新的變數 Survived_cate，資料型態傳換成類別型態      
# 2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [4]:
# 取出資料後，把遺失值刪除
data = df_train.dropna()

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
data = data.drop(['Name','Ticket','PassengerId'], axis=1)
display(data.head(5))

print(data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [5]:
num_features = [feature for dtype, feature in zip(data.dtypes,data.columns)\
                if dtype == 'float64' or dtype == 'int64']
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [6]:
cat_features = [feature for dtype, feature in zip(data.dtypes, data.columns)\
                if dtype == 'object']
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



## 定義 x,y

In [7]:
x = data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y = data['Survived']
display(x.head(5))
display(y.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

### Q1: 目標變數為 Survived，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!


In [8]:
# 離散要轉換成數值，['Sex', 'Embarked']
# Can only use .cat accessor with a 'category' dtype
# .cat.codes ：分類的編碼，用數字表示分類數據

for col in ['Sex', 'Embarked']:
    c = x.loc[:, col].astype('category')
    _dict = dict(enumerate(c.cat.categories))
    print (f'{col}: {_dict}')
    x.loc[:, col] = c.cat.codes # ok
display(x.head(5))

Sex: {0: 'female', 1: 'male'}
Embarked: {0: 'C', 1: 'Q', 2: 'S'}


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2


In [9]:
#執行 包裝法
estimator = SVC(kernel='linear')
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(x, y)

# True = selected feature
# print(selector.support_) 
# Selected (i.e., estimated best) features are assigned rank 1.
# print(selector.ranking_)

rfe_feature = ','.join(x.loc[:,selector.support_].columns.tolist())
print(f'被選中的特徵: {rfe_feature}')

print('特徵排名: ')
for pair in sorted(list(zip(selector.ranking_,x.columns.tolist()))):
    print('\t',pair[0],pair[1])

被選中的特徵: Sex
特徵排名: 
	 1 Sex
	 2 Embarked
	 3 Pclass
	 4 SibSp
	 5 Age
	 6 Parch
	 7 Fare
